In [5]:
import pandas as pd
from gemini import  clean_user_answer
from models import variable_to_question, FormattedResponse
from typing import  List


# Limpieza

In [6]:
data = pd.read_csv('data/encuesta_raw.csv')
# remover respuesta de prueba
data = data[~(data['Id'] == 1)]
# remover columnas innecesaias
data.drop(['Email', 'Name'], axis=1, inplace=True)

# renombrar las columnas
data.rename(columns={
    'Id': 'response_id',
    'Start time': 'start_response',
    'Completion time': 'complete_response',
    'Edad': 'age',
    'Sexo': 'sex',
    'Año de estudio en el área de la salud.': 'years_studying',
    'Que carrera esta cursando?': 'degree',
    '¿Cómo calificaría el estado actual de la infraestructura hospitalaria en Managua?': 'infrastructure_score',
    '¿Ha notado mejoras en la infraestructura hospitalaria durante el período 2020–2025?': 'notice_infrastructure_improvement',
    'Mencione almenos 3 cambios en la infraestructura hospitalaria que considere relevante que haya llevado a cabo nuestro gobierno revolucionario.': 'infrastructure_changes',
    '¿Con qué frecuencia ha observado el uso de  sistemas digitales (expedientes electrónicos, telemedicina, imágenes  digitales) durante sus prácticas o rotaciones?': 'frecuency_digital_systems',
    '¿Cree que la incorporación de nuevas tecnologías ha mejorado la atención a los pacientes?': 'technology_improves_attention',
    'Mencione una tecnología hospitalaria que considere más importante en la actualidad.': 'most_important_technologies',
    'En su experiencia, ¿la modernización hospitalaria ha mejorado la calidad de la atención?': 'modernization_improves_attention',
    '¿Qué aspecto considera que ha mejorado más? ': 'improved_aspects',
    '¿Qué desafíos persisten en la atención hospitalaria a pesar de la modernización?': 'challenges',
    '¿Cuáles son las patologías más frecuentes que ha observado en sus prácticas?': 'pathologies',
    '¿Considera que la red hospitalaria está mejor preparada para atender estas patologías en comparación con hace 5 años?': 'improved_pathologies_treatments',
    'En general, ¿cómo calificaría la modernización de la red hospitalaria pública en Managua (2020–2025)?  ': 'modernization_score',
    '¿Qué recomendación daría para mejorar la modernización hospitalaria en Nicaragua?': 'recomendations'
}, inplace=True)

data['degree'] = data['degree'].replace(
    'Doctor en Medicina y Cirugía', 'Medicina y Cirugía')

# Normalizar el texto de las variables abiertas
def normalize_text(text: str):
    if pd.isna(text):
        return text
    text = text.lower().strip()
    text = ' '.join(text.split())
    return text


data["infrastructure_changes"] = data["infrastructure_changes"].apply(
    normalize_text)
data["most_important_technologies"] = data["most_important_technologies"].apply(
    normalize_text)
data["challenges"] = data["challenges"].apply(normalize_text)
data["recomendations"] = data["recomendations"].apply(normalize_text)


# corregir tipos de datos
data = data.astype({
    'response_id': 'string',
    'start_response': 'datetime64[ns]',
    'complete_response': 'datetime64[ns]',
    'age': 'category',
    'sex': 'category',
    'years_studying': 'category',
    'degree': 'category',
    'infrastructure_score': 'category',
    'notice_infrastructure_improvement': 'category',
    'frecuency_digital_systems': 'category',
    'technology_improves_attention': 'category',
    'modernization_improves_attention': 'category',
    'improved_pathologies_treatments': 'category',
    'pathologies': 'string',
    'modernization_score': 'int64',

    'infrastructure_changes': 'string',
    'most_important_technologies': 'string',
    'improved_aspects': 'string',
    'challenges': 'string',
    'recomendations': 'string'
})

# Limpiar caracteristicas mal formateadas
data['age'] = data['age'].str.replace('–', '-').astype('category')
data['notice_infrastructure_improvement'] = data['notice_infrastructure_improvement'].str.replace('No estoyy seguro/a', 'No estoy seguro/a').astype('category')
data['pathologies'] = data['pathologies'].str.replace('Crónicas (ej. diabetes, hipertensión)', 'Crónicas')
data["improved_pathologies_treatments"] = data["improved_pathologies_treatments"].str.replace('No estoy seguro/aOption 3', 'No estoy seguro/a').astype('category')

# formatear listas
data['pathologies'] = data['pathologies']\
    .apply(lambda x: [y for y in x.split(';') if len(y) > 0] )\
    .apply(lambda x: [y if y in ['Respiratorias', 'Cardiovasculares', 'Infecciosas', 'Traumáticas', 'Crónicas'] else 'Otro' for y in x])
data['improved_aspects'] = data['improved_aspects']\
    .apply(lambda x: [y for y in x.split(';') if len(y) > 0])\
    .apply(lambda x: [y if y in ['Tiempos de espera', 'Disponibilidad de equipos', 'Comodidad para pacientes', 'Acceso a expedientes digitales'] else 'Otro' for y in x])

# Dar orden a las variables categoricas
data['age'] = data['age'].cat.set_categories(['Menos de 17 años','17-19 años', '23-25 años', '26 años o más'], ordered=True)
data['years_studying'] = data["years_studying"].cat.set_categories(list(range(1,6)), ordered=True)
data["infrastructure_score"] = data["infrastructure_score"].cat.set_categories(['Muy deficiente', 'Deficiente', 'Regular' ,'Buena',  'Muy buena', ], ordered=True)
data["frecuency_digital_systems"] = data["frecuency_digital_systems"].cat.set_categories(['Nunca', 'Rara vez', 'A veces', 'Frecuentemente', 'Muy frecuentemente'], ordered=True)
data["technology_improves_attention"] = data["technology_improves_attention"].cat.set_categories(['No estoy seguro/a', 'No ha mejorado', 'Sí, de manera parcial', 'Sí, significativamente'] ,ordered=True)
data["modernization_improves_attention"] = data["modernization_improves_attention"].cat.set_categories(['No', 'No estoy seguro/a', 'Parcialmente', 'Si'], ordered=True)
data["improved_pathologies_treatments"] = data["improved_pathologies_treatments"].cat.set_categories(['No estoy seguro/a', 'No', 'Si'], ordered=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110 entries, 1 to 110
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   response_id                        110 non-null    string        
 1   start_response                     110 non-null    datetime64[ns]
 2   complete_response                  110 non-null    datetime64[ns]
 3   age                                110 non-null    category      
 4   sex                                110 non-null    category      
 5   years_studying                     110 non-null    category      
 6   degree                             110 non-null    category      
 7   infrastructure_score               110 non-null    category      
 8   notice_infrastructure_improvement  110 non-null    category      
 9   infrastructure_changes             110 non-null    string        
 10  frecuency_digital_systems          110 non-

## Agregar categorias a las carreras

In [7]:
DEGREE_CATEGORY_MAP = {
    # Personal Médico y Quirúrgico
    "Medicina y Cirugía": "Personal Médico y Quirúrgico",
    "Licenciatura en Anestesia y Reanimación": "Personal Médico y Quirúrgico",

    # Personal de Enfermería
    "Licenciatura en Enfermería": "Personal de Enfermería",

    # Personal de Diagnóstico y Laboratorio
    "Licenciatura en Bioanálisis Clínico": "Personal de Diagnóstico y Laboratorio",
    "Licenciatura en Microbiología": "Personal de Diagnóstico y Laboratorio",
    "Técnico Superior en Citología Cervical": "Personal de Diagnóstico y Laboratorio",

    # Personal de Terapia y Rehabilitación
    "Licenciatura en Fisioterapia": "Personal de Terapia y Rehabilitación",
    "Licenciatura en Nutrición": "Personal de Terapia y Rehabilitación",

    # Personal de Atención Especializada
    "Odontología": "Personal de Atención Especializada",
    "Licenciatura en Optometría Médica": "Personal de Atención Especializada",
    "Podología": "Personal de Atención Especializada",

    # Personal de Salud Pública y Prevención
    "Técnico Superior en Higiene y Epidemiología": "Personal de Salud Pública y Prevención"
}


data['degree_category'] = data['degree'].apply(
    lambda x: DEGREE_CATEGORY_MAP[x]).astype('category')

# Limpiar campos de preguntas abiertas

In [8]:
open_question_variables = [
    'infrastructure_changes',
    'most_important_technologies',
    'challenges',
    'recomendations'
]
open_questions_df = data.loc[:, ["response_id"]]
BATCH_SIZE = 65

for  open_question_variable in open_question_variables:
    print("Starting formatting of variable: ", open_question_variable)
    items_ready: List[FormattedResponse] = []
    batch_number = 1

    for start in range(0, len(data), BATCH_SIZE):
        batch = data[["response_id", open_question_variable]].iloc[start : start + BATCH_SIZE]
        fixed_batched = clean_user_answer(variable_to_question[open_question_variable], batch.to_dict('records'))

        items_ready += fixed_batched
        print("Finished batch: ", batch_number)
        batch_number+=1

    result_df = pd.DataFrame([x.model_dump() for x in items_ready])

    result_df["sentiment"] = result_df["sentiment"].astype('category')

    result_df.columns = ['response_id', open_question_variable, open_question_variable+'_sentiment']

    open_questions_df = open_questions_df.merge(result_df, how='left', on='response_id')


data = data.drop(columns=open_question_variables).merge(open_questions_df, on="response_id")

Starting formatting of variable:  infrastructure_changes
Finished batch:  1
Finished batch:  2
Starting formatting of variable:  most_important_technologies
Finished batch:  1
Finished batch:  2
Starting formatting of variable:  challenges
Finished batch:  1
Finished batch:  2
Starting formatting of variable:  recomendations
Finished batch:  1
Finished batch:  2


# Save data

In [9]:
data.to_parquet('data/01_Encuesta.parquet')